# April 2021 - Challenge

Original problem statement: https://www.research.ibm.com/haifa/ponderthis/challenges/May2021.html

## Problem statement

The Fibonacci sequence is defined by  $F_0 = 0, F_1 = 1$ and the recurrence relationship $F_n = F_{n-1} + F_{n-2}$

We call any sequence  $A_0, A_1, A_2,\ldots$ Fibonacci-like if it satisfies the recurrences $A_n = A_{n-1}+A_{n-2}$ for all  $n \ge 2$.

The Fibonacci sequence contains many prime numbers. For instance,  $F_3 = 2, F_{11} = 89$, etc. Our goal is to see how to generate a fibonnaci-like sequence without any prime numbers and relatively prime initial elements. This amounts to finding suitable relatively prime  $A_0$ and  $A_1$.

The main step in the generation is finding a set [(p_1, m_1, a_1), (p_2, m_2, a_2),..., (p_t, m_t, a_t)] of triplets of the form (p_k, m_k, a_k) such that:

1. $1 \le a_k \le m_k$
2. For every natural number $n$, we have that for some $k$, $n$ is equivalent to  $a_k modulo  m_k$ (i.e. $m_k$ divides  $n-a_k$).
3. $p_k$ is a prime divisor of the Fibonacci number  $F_{m_k}$
4. All the  $p_k$ are distinct (the  m_k and  a_k can be non-distinct)

Given this set, one can generate  $A_0$ and  $A_1$ that satisfy

 $A_0$ is equivalent to $F_{m_k-a_k}$ modulo  $p_k$
 $A_1$ is equivalent to $F_{m_k-a_k+1}$ modulo  $p_k$
 
and one can prove that this sequence does not contain any primes by using the following easy-to-prove identity, which holds for any Fibonacci-like sequence:  $A_{m+n} = A_mF_{n-1} + A_{m+1}F_n$.

Your goal is to find the set [(p_1, m_1, a_1),..., (p_t, m_t, a_t)] of triplets satisfying conditions 1-4 described above. (Hint: A set of 18 elements exists where the only primes dividing its  $m_k$ elements are 2,3 and 5).

A bonus "*" will be given for computing  $A_0$ and  $A_1$ from the found set, and explaining why every element of  $A_n$ is divisible by one of the  $p_k$ elements, but not equal to it.

## Solution approach:

The problem can be stated as a MIP. There are different ways of modelling it, but not every way is efficient. For example, I first tried to have binary variables for each possible p, m and a for every possible t (assuming a fixed size of |t|). This turned out not to be efficient enough. An approach where all possible combinations of p, m and a are combined into individual binary variables works better.

Condition 1 and 3 can already be applied when the variables are set up. Condition 4 is very simple to express.
Condition 2 is a bit more tricky. Unfortunately, it isn't possible to explicitiely formulate it for every possible $n$, but it seems efficient to just do it for a large enough number (2000 in my case). Also, in order to keep the number of variables limited, we need to make some assumptions about the maximum size of m, a and p. It turned out to work with a relatively limited solution space, but this might not always be the case, so maybe we were a bit lucky.

Since condition 2 is not stated explicitely for every possible $n$, we have to check it afterwards (for a large enough testset of $n$).

In [1]:
from docplex.mp.model import Model
import math
from sympy import factorint

def primefactors(n):
    res = set()
    for i in factorint(n):
        res.add(i)
    return res

def isprime(n):
    pfs = primefactors(n)
    return len(pfs) == 1 and list(pfs)[0] == n

T = 18
ts = range(T)
max_mk = 340 # we are just assuming this is big enough
possible_mks = list(range(2,max_mk))
possible_mks = [i for i in possible_mks if max(primefactors(i)) <= 5] # Taking advantage of the fact that we know that there is a solution where each m_k has no other primefactors than 2, 3 and 5
max_ak = 340 # we are just assuming this is big enough
possible_aks = list(range(1,max_ak))
max_pk = 5000 # we are just assuming this is big enough
possible_pks = [i for i in range(2,max_pk) if isprime(i)]
f = [0,1]
for i in range(100000):
    f.append(f[-2]+f[-1])

model = Model()

print('Set up variables')
pma = dict()
for m in possible_mks:
    fm = f[m]
    pfs = primefactors(fm)
    for a in possible_aks:
        if a <= m:
            for p in possible_pks:
                if p > fm:
                    break
                if p in pfs:
                    pma[(p,m,a)] = model.binary_var()
          
# Set size
print('Set size constraint')
model.add(model.sum(pma.values()) == T)
    
# 1. 1 <= a_k <= m_k
# part of variable definition
        
# 2. For every natural number n, we have that for some k, n is equivalent to  a_k modulo  m_k (i.e.  m_k divides  n-a_k ).
print('Adding constraint 2')
#for n in list(range(0,60))+list(range(110,125))+list(range(455,470)):
for n in range(0,2000):
    #print(n)
    possible_options = []
    for p,m,a in pma.keys():
        if (n-a)/m == int((n-a)/m):
            possible_options.append(pma[(p,m,a)])
    model.add(model.sum(possible_options) >= 1)

# 3. p_k is a prime divisor of the Fibonacci number  F_{m_k}
# part of variable definition

# 4. All the  p_k are distinct (the  m_k and  a_k can be non-distinct)
print('Adding constraint 4')
for p2 in possible_pks:
    model.add(model.sum(pma[(p,m,a)] for p,m,a in pma.keys() if p == p2) <= 1)
    
model.solve(log_output = True)
model.export_as_lp('model.lp')

answers = []
for p,m,a in pma.keys():
    if pma[(p,m,a)].solution_value > 0.5:
        answers.append((p,m,a))
print(answers)

unsatisfied = []
for n in range(1000000):
    good = False
    for t in ts:
        if (n-answers[t][2])/answers[t][1] == int((n-answers[t][2])/answers[t][1]):
            good = True
            #print('%i-%i can be divided by %i'%(n, answers[t][2] , answers[t][1]))
            break
    if not good:
        #print('%i doesnt satisfy condition 2!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'%n)
        unsatisfied.append(n)
print(unsatisfied)
print('Condition 2 unsatisfied for %i numbers'%len(unsatisfied))

Set up variables
Set size constraint
Adding constraint 2
Adding constraint 4
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 1 time.
MIP Presolve eliminated 628 rows and 17014 columns.
Reduced MIP has 2042 rows, 36518 columns, and 519436 nonzeros.
Reduced MIP has 36518 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.55 sec. (365.28 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 11581 columns.
Reduced MIP has 2042 rows, 24937 columns, and 347154 nonzeros.
Reduced MIP has 24937 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.27 sec. (217.52 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 7140 columns.
Reduced MIP has 2042 rows, 17797 columns, and 251570 nonzeros.
Reduced MIP has 17797 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.15 sec. (147.86 ticks)
Tried aggreg

There we go. I found and submitted the following solution:

[(2, 3, 1), (3, 4, 3), (5, 5, 4), (7, 8, 5), (17, 9, 6), (11, 10, 2), (61, 15, 8), (47, 16, 9), (19, 18, 18), (41, 20, 20), (23, 24, 17), (31, 30, 26), (2161, 40, 30), (1103, 48, 33), (541, 90, 48), (181, 90, 66), (2521, 120, 50), (241, 120, 90)]

I double checked this for very high $n$ and it seemed to be correct.

I didn't solve the bonus part of the question due to time restrictions and since I was lacking the right idea.